# Свойства многофазного потока по uniflocvba и uniflocpy

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uTools.data_workflow as data_workflow
import pandas as pd
from uniflocpy.uTools import plotly_workflow
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uTools.uconst as uc
import uniflocpy.uPVT.PVT_fluids as PVT_fluids

In [ ]:
def create_result_df_from_vba_output(vba_result, p_bar, t_c):
    result_dict = {}
    for key, value in zip(vba_result[1],vba_result[0]):
        result_dict[key] = value
    result_dict['p_bar'] = p_bar
    result_dict['t_c'] = t_c
    this_df = pd.DataFrame(result_dict, index = [0])
    return this_df

Загрузка надстройки

In [ ]:
uniflocvba = python_api.API('E:\\Git\\unifloc_vba\\UniflocVBA_7.xlam')

Задание параметров

In [ ]:
gamma_oil =0.86
gamma_water = 1
gamma_gas =  1.45/1.293
rsb_m3m3 = 56
t_res_c = 20
t_c = 20
p_bar = 90
q_liq_sm3day = 40
watercut_perc = 0


keywords_python = {"gamma_oil": gamma_oil, "gamma_gas": gamma_gas, "gamma_wat":gamma_water,
                                    "rsb_m3m3": rsb_m3m3, "t_res_c": t_res_c, 'pb_bar':90}

keywords_vba = {"qliq_sm3day":q_liq_sm3day, "fw_perc":watercut_perc,
                "t_C": t_c, "gamma_gas": gamma_gas,
                "gamma_oil": gamma_oil, "gamma_wat": gamma_water, "rsb_m3m3": rsb_m3m3, "tres_C": t_res_c}



Настройка модели флюида

In [ ]:
blackoil_option = BlackOil_model.BlackOil_option()
blackoil_option.b_wat_cor_number = 1
blackoil_option.mu_wat_cor_number = 1
blackoil_option.rho_wat_cor_number = 1
blackoil_option.z_cor_number = 1
blackoil_option.pseudocritical_temperature_cor_number = 1
blackoil_option.pseudocritical_pressure_cor_number = 1
blackoil_option.rho_gas_cor_number = 1
blackoil_option.b_gas_cor_number = 1 - 1
blackoil_option.mu_dead_oil_cor_number = 2
blackoil_option.sigma_oil_gas_cor_number = 2
blackoil_option.sigma_wat_gas_cor_number = 1

#blackoil_option.rs_cor_number =2
blackoil_option.pb_cor_number = 3

blackoil_option.set_vba_preset()


python_fluid = BlackOil_model.Fluid(**keywords_python, option = blackoil_option)

Инициализация многофазного потока

In [ ]:
multiphase_flow = PVT_fluids.FluidFlow(python_fluid)
multiphase_flow.qliq_on_surface_m3day = q_liq_sm3day
multiphase_flow.fw_on_surface_perc  =watercut_perc

In [ ]:
import scipy.optimize as sp  # модуль для решения уравения
import numpy as np
def coef_mt(t_k, rho_deadoil_kgm3, gamma_gas):
    return 1 + 0.029 * (t_k - 293) * (rho_deadoil_kgm3 * gamma_gas * 10 ** (-3) - 0.7966)
def coef_dt(t_k, rho_deadoil_kgm3, gamma_gas):
    return 10 ** (-3) * rho_deadoil_kgm3 * gamma_gas * (4.5 - 0.00305 * (t_k - 293)) - 4.785
def coef_rp(p_mpa, pb_mpa):
    return (1 + np.log10(p_mpa)) / (1 + np.log10(pb_mpa)) - 1

def calc_gas(t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, rsb_m3m3, return_m3m3 = True):
    
    rsb_m3t = rsb_m3m3 / gamma_oil
    rp = coef_rp(p_mpa, pb_mpa)
    #print(f"rp={rp}")
    dt = coef_dt(t_k, rho_deadoil_kgm3, gamma_gas)
    #print(f"dt={dt}")
    mt = coef_mt(t_k, rho_deadoil_kgm3, gamma_gas)
    #print(f"mt={mt}")
    gas_liberated_m3t = rsb_m3t * rp * mt * (dt*(1+rp)-1)
    gas_dissolved_m3t = rsb_m3t * mt - gas_liberated_m3t
    
    if p_mpa >= pb_mpa:
        if return_m3m3:
            return 0, rsb_m3t * mt * rho_deadoil_kgm3 / 1000
        else:
            return 0, rsb_m3t * mt
            
    if return_m3m3:
        return gas_liberated_m3t * rho_deadoil_kgm3 / 1000, gas_dissolved_m3t * rho_deadoil_kgm3 / 1000
        #return gas_liberated_m3t, gas_dissolved_m3t
    else:
        return gas_liberated_m3t, gas_dissolved_m3t

def calc_gas_for_fsolve(rsb_m3m3_real, t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, rsb_m3m3, return_m3m3 = True):
    gas_dissolved_m3m3 =calc_gas(t_k, rho_deadoil_kgm3, gamma_gas, pb_mpa, pb_mpa, rsb_m3m3_real, return_m3m3 = True)[1]
    return (gas_dissolved_m3m3 - rsb_m3m3)**2

def calc_gas_with_fsolve(t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, rsb_m3m3, return_m3m3 = True):
    method = 'excitingmixing'
    answer = sp.root(calc_gas_for_fsolve, rsb_m3m3, args = (t_k, rho_deadoil_kgm3, gamma_gas,
                                                            pb_mpa, pb_mpa, rsb_m3m3, return_m3m3), tol = 0.01,
                    options = {'xtol':0.01, 'maxfev': 25, 'eps': 0.01})
    #print(answer)
    answer = answer.x[0]
    
    return calc_gas(t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, answer, return_m3m3)




In [ ]:
t_k = 300.5
p_mpa = 5.5
p_res_mpa =17.5
t_res_k = 313
gamma_oil = 0.868
rsb_m3t = 55.6
p_b_mpa = 9.2
gamma_gas = 1.119
y_a = 0.069
u_c1 = 0.355

In [ ]:
def calc_pb(pb_mpa, rho_deadoil_kgm3, rsb_m3t, tres_k, t_k, ch4_d = 0.4, n2_d=0.08):
    #gi = rsb_m3m3 * 10**3 * 273 / 293 / rho_deadoil_kgm3
    pbt_mpa = pb_mpa - (tres_k - t_k) / (9.157 + 701.8 / (rsb_m3t * (ch4_d - 0.8* n2_d)))
    return pbt_mpa
    

In [ ]:
p_b_t_mpa = calc_pb(p_b_mpa, gamma_oil * 1000, rsb_m3t, t_res_k, t_k,u_c1 , y_a)
p_b_t_mpa

In [ ]:
gas_liberated_m3t, gas_dissolved_m3t =  calc_gas(t_k, gamma_oil * 1000, gamma_gas, p_mpa, p_b_t_mpa, rsb_m3t * gamma_oil, False)
gas_liberated_m3t, gas_dissolved_m3t

In [ ]:
def rho_gas_liberated_relative(p_mpa, pb_mpa, gamma_gas, gamma_oil, t_k, rsb_m3m3):
    rsb_m3t = rsb_m3m3 / gamma_oil
    
    a = 1 + 0.0054*(t_k - 293)
    
    u = gamma_oil * rsb_m3t - 186
    rp = coef_rp(p_mpa, pb_mpa)
    result = a * (gamma_gas - 0.0036 * (1 +  rp) * (105.7 + u * rp))
    return result

In [ ]:
rho_gas_liberated_d = rho_gas_liberated_relative(p_mpa, p_b_t_mpa, gamma_gas,
                                                 gamma_oil, t_k , rsb_m3t * gamma_oil)
rho_gas_liberated_d

In [ ]:
def rho_gas_dissolved_relative(rsb_m3t,gamma_oil,gamma_gas,
                               gas_liberated_m3t, gas_dissolved_m3t, rho_gas_liberated_d,
                              p_mpa, pb_mpa, t_k):
    
    a = 1 + 0.0054*(t_k - 293)
    #print(a)
    mt = coef_mt(t_k, gamma_oil*1000, gamma_gas)
    #print(mt)
    rp = coef_rp(p_mpa, pb_mpa)
    #print(rp)
    result = rsb_m3t * (a * mt * gamma_gas -
                        rho_gas_liberated_d * gas_liberated_m3t/rsb_m3t)/gas_dissolved_m3t
    #print(rsb_m3t,a,  mt,gamma_gas, rho_gas_liberated_d, gas_liberated_m3t,  rsb_m3t,gas_dissolved_m3t)
    return result

In [ ]:
rho_gas_dissolved_relative_d = rho_gas_dissolved_relative(rsb_m3t, gamma_oil,gamma_gas, 
                                                          gas_liberated_m3t, gas_dissolved_m3t, 
                                                          rho_gas_liberated_d,
                                                         p_mpa, p_b_t_mpa, t_k)
rho_gas_dissolved_relative_d

In [ ]:
gas_dissolved_m3t

In [ ]:
def b_oil_below_pb_m3m3(t_k,p_mpa, gamma_oil,gamma_gas, 
          
          rho_gas_dissolved_relative_d, rho_gas_liberated_relative,
          
         gas_dissolved_m3t):
    
    a = 1 + 0.0054*(t_k - 293)
    #print(a)
    mt = coef_mt(t_k, gamma_oil*1000, gamma_gas)
    #print(mt)

    lambda_t = 10**(-3) * (4.3 - 3.54 * gamma_oil + (1.0337 * rho_gas_dissolved_relative_d) / a + \
                5.581 * 10**(-3) * gamma_oil * (1 - 1.61 * 10**(-3) * gas_dissolved_m3t) * \
    gas_dissolved_m3t)
    #print(lambda_t)
    if 0.78 < gamma_oil <= 0.860:
        alpha_n = 10**(-3)*(3.083-2.638*gamma_oil)
    else:
        alpha_n = 10**(-3)*(2.513-1.975*gamma_oil)
        print(alpha_n)
        
    result = 1 + 1.0733 * gamma_oil * gas_dissolved_m3t * lambda_t / mt + alpha_n * (t_k - 293) - \
    6.5 * 10**(-4) * p_mpa
    return result

In [ ]:
b_oil_m3m3 = b_oil_below_pb_m3m3(t_k, p_mpa, gamma_oil, gamma_gas, rho_gas_dissolved_relative_d, rho_gas_liberated_d,
                   gas_dissolved_m3t)
b_oil_m3m3

In [ ]:
def unf_rho_oil_kgm3(gamma_oil, rho_gas_dissolved_relative_d, 
                 gas_dissolved_m3t, t_k, gamma_gas, b_oil_below_pb_m3m3):
    a = 1 + 0.0054*(t_k - 293)
    #print(a)
    mt = coef_mt(t_k, gamma_oil*1000, gamma_gas)
    #print(mt)
    result = gamma_oil * 1000 * (1 + 
                                 1.293 * 10**(-3) *  rho_gas_dissolved_relative_d *gas_dissolved_m3t / \
                                (a * mt) ) / b_oil_below_pb_m3m3
    return result
    

In [ ]:
rho_oil_kgm3 = unf_rho_oil_kgm3(gamma_oil, rho_gas_dissolved_relative_d, gas_dissolved_m3t,
                               t_k, gamma_gas, b_oil_m3m3)
rho_oil_kgm3

In [ ]:
def unf_mu_dead_oil_293_cp(gamma_oil):
    rho_oil_stkgm3 = gamma_oil * 1000
    if 0.845 < gamma_oil < 0.924:
        return ( (0.658 * rho_oil_stkgm3**2) / (886 * 10**3 - rho_oil_stkgm3 **2) )**2
    else:
        return ( (0.456 * rho_oil_stkgm3**2) / (883 * 10**3 - rho_oil_stkgm3 **2) )**2


In [ ]:
mu_dead_oil_293_cp = unf_mu_dead_oil_293_cp(gamma_oil)
mu_dead_oil_293_cp

In [ ]:
import math

In [ ]:
def unf_mu_dead_oil_t_cp(mu_dead_oil_293_cp, t_k, gamma_oil):
    a = 10 ** (-0.0175 * (293 - t_k) - 2.58)
    b = (8 * 10**(-5) * gamma_oil * 1000 - 0.047) * mu_dead_oil_293_cp ** (0.13 + 0.002 * (t_k - 293))
    result = mu_dead_oil_293_cp * ((293 - t_k)**a) * (math.exp(b*(293-t_k)))
    if type(result) == complex:
        result = result.real
    return result

In [ ]:
mu_dead_oil_t_cp = unf_mu_dead_oil_t_cp(mu_dead_oil_293_cp, t_k, gamma_oil)
mu_dead_oil_t_cp

In [ ]:
def unf_mu_oil_below_pb_cp(mu_dead_oil_t_cp, gas_dissolved_m3t, gamma_oil):
    if 0.78 < gamma_oil <= 0.860:
        alpha_n = 10**(-3)*(3.083-2.638*gamma_oil)
    else:
        alpha_n = 10**(-3)*(2.513-1.975*gamma_oil)
    gas_dissolved_m3t_in_dif_cond = 1.055 * (1 + 5 * alpha_n) * gas_dissolved_m3t * gamma_oil
    a = 1 + 0.0129 * gas_dissolved_m3t_in_dif_cond - 0.0364 * gas_dissolved_m3t_in_dif_cond ** 0.85
    b = 1 + 0.0017 * gas_dissolved_m3t_in_dif_cond - 0.0228 * gas_dissolved_m3t_in_dif_cond ** 0.667
    mu_oil = a * mu_dead_oil_t_cp ** b
    return mu_oil

In [ ]:
mu_oil_below_pb_cp = unf_mu_oil_below_pb_cp(mu_dead_oil_t_cp, gas_dissolved_m3t, gamma_oil)
mu_oil_below_pb_cp

In [ ]:
def unf_sigma_oil_gas_nm(p_mpa, t_k, gamma_gas):
    return (1/10) **(1.58 + 0.05*p_mpa) - 72 * 10**(-6)*(t_k-305)

In [ ]:
sigma_oil_gas_nm = unf_sigma_oil_gas_nm(p_mpa, t_k, gamma_gas)
sigma_oil_gas_nm

Расчет нефти в пластовых условиях

In [ ]:
t_k = 313
p_mpa = 17.5

In [ ]:
def unf_rs_res_m3t(rsb_m3t, gamma_oil,gamma_gas, t_k):
    mt = coef_mt(t_k, gamma_oil*1000, gamma_gas)
    return rsb_m3t * mt

In [ ]:
rs_res_m3t = unf_rs_res_m3t(rsb_m3t, gamma_oil,gamma_gas, t_k)
rs_res_m3t

In [ ]:
def unf_alpha_n(gamma_oil):
    if 0.78 < gamma_oil <= 0.860:
        alpha_n = 10**(-3)*(3.083-2.638*gamma_oil)
    else:
        alpha_n = 10**(-3)*(2.513-1.975*gamma_oil)
    return alpha_n
    

In [ ]:
def unf_rho_gas_relative_in_oil_res_d(rsb_m3t, rs_res_m3t, t_k, gamma_gas):
    a = 1 + 0.0054 * (t_k - 293)
    mt = coef_mt(t_k, gamma_oil*1000, gamma_gas)
    return a * mt * gamma_gas * rsb_m3t / rs_res_m3t
    

In [ ]:
rho_gas_relative_in_oil_res_d = unf_rho_gas_relative_in_oil_res_d(rsb_m3t, rs_res_m3t, t_k, gamma_gas)
rho_gas_relative_in_oil_res_d

In [ ]:
def unf_b_oil_res_m3m3(gamma_oil,gamma_gas, p_mpa, rs_res_m3t, rho_gas_relative_dissolved_d, rsb_m3t):
    a = 1 + 0.0054 * (t_k - 293) #TODO check
    #print(a)
    alpha_n = unf_alpha_n(gamma_oil)
    #print(alpha_n)
    mt = coef_mt(t_k, gamma_oil*1000, gamma_gas)
    lambda_coef = (10**(-3) * (4.3 - 3.54 * gamma_oil + 1.0337 * rho_gas_relative_dissolved_d / a + 
                              5.581 * 10**(-3) * gamma_oil * 
                               (1 - 1.61 * 10**(-3) * gamma_oil * rs_res_m3t )/
                               (rs_res_m3t)))
    #print(lambda_coef)
    #print(rs_res_m3t)
    result = (1 + 
             1.0733 * gamma_oil * lambda_coef * rs_res_m3t / mt + 
             alpha_n * (t_k - 293) - 
             6.5*10**(-4)*p_mpa)
    return result
    
    


In [ ]:
b_oil_res_m3m3 = unf_b_oil_res_m3m3(gamma_oil,gamma_gas,  p_mpa,rs_res_m3t, rho_gas_relative_in_oil_res_d, rsb_m3t)
b_oil_res_m3m3=1.146
b_oil_res_m3m3

In [ ]:
def unf_rho_oil_above_pb_m3m3(rs_res_m3t, gamma_gas, gamma_oil,
                          t_k, rho_gas_relative_dissolved_d, b_oil_res_m3m3):
    a = 1 + 0.0054 * (t_k - 293)
    mt = coef_mt(t_k, gamma_oil*1000, gamma_gas)


    result = (gamma_oil * 1000 * 
             (1 + 1.293 * 10**(-3) * rho_gas_relative_dissolved_d * rs_res_m3t / (a * mt))) / b_oil_res_m3m3
    return result

In [ ]:
rho_oil_above_pb_m3m3 = unf_rho_oil_above_pb_m3m3(rs_res_m3t, gamma_gas, gamma_oil,
                          t_k, rho_gas_relative_in_oil_res_d, b_oil_res_m3m3)
rho_oil_above_pb_m3m3

In [ ]:
def unf_mu_oil_above_pb_cp(gamma_oil, rsb_m3t, t_k, p_mpa, p_b_mpa):

    mu_dead_oil_293_cp = unf_mu_dead_oil_293_cp(gamma_oil)
    print(mu_dead_oil_293_cp)
    
    mu_dead_oil_t_cp = unf_mu_dead_oil_t_cp(mu_dead_oil_293_cp, t_k, gamma_oil)
    #print(mu_dead_oil_t_cp)
    
    alpha_n = unf_alpha_n(gamma_oil)
    g_with_star = 1.055 * ( 1 + 5 *alpha_n) * rsb_m3t * gamma_oil
    #print(g_with_star)
    a = 1 + 0.0129 * g_with_star - 0.0364 * g_with_star ** 0.85
    #print(a)
    b = 1 + 0.0017 * g_with_star - 0.0228 * g_with_star ** 0.667
    #print(b)
    
    mu_oil_pb_cp = a * mu_dead_oil_t_cp ** b
    #print(mu_oil_pb_cp)
    
    
    if mu_oil_pb_cp < 5:
        delta = 0.0114 * mu_oil_pb_cp
    elif 5<=mu_oil_pb_cp < 10:
        delta = 0.0114 * (mu_oil_pb_cp - 5)
    elif 10<=mu_oil_pb_cp < 25:
        delta = 0.0171 + 0.031 * (mu_oil_pb_cp -10)
    elif 25<=mu_oil_pb_cp < 45:
        delta = 0.0171 + 0.031 * (mu_oil_pb_cp -10)
    elif 45<=mu_oil_pb_cp < 75:
        delta = 0.0171 + 0.031 * (mu_oil_pb_cp -10)
    #elif 75<=mu_oil_pb_cp < 85:
    else:
        delta = 0.0171 + 0.031 * (mu_oil_pb_cp -10)
    #print(delta)
    result = mu_oil_pb_cp + delta * (p_mpa - p_b_mpa)
    #print(mu_oil_pb_cp, delta,p_mpa, p_b_mpa)
    
    return result

    

In [ ]:
mu_oil_above_pb_cp = unf_mu_oil_above_pb_cp(gamma_oil, rsb_m3t, t_k, p_mpa, p_b_mpa)
mu_oil_above_pb_cp

In [ ]:
rho_oil_kgm3 = unf_rho_oil_kgm3(gamma_oil, 1.2356, 49.02,
                               t_k, gamma_gas, 1.120)
rho_oil_kgm3

In [ ]:
calc_gas(300.5, 868, 1.119, 5.5, 8.95, 55.6)

In [ ]:
calc_gas_with_fsolve(300.5, 868, 1.119, 5.5, 8.95, 55.6)

In [ ]:
rus_fluid_flow = BlackOil_model.Fluid(**keywords_python, activate_rus_cor=1, p_reservoir_bar=100)

In [ ]:
multiphase_flow2 = PVT_fluids.FluidFlow(rus_fluid_flow)
multiphase_flow2.qliq_on_surface_m3day = q_liq_sm3day
multiphase_flow2.fw_on_surface_perc  =watercut_perc

In [ ]:
rus_fluid_flow = multiphase_flow2

Расчет

In [ ]:
python_flow_model_db = data_workflow.Data()
python_flow_model_db.clear_data()



rus_flow_model_db = data_workflow.Data()
rus_flow_model_db.clear_data()

vba_result_df = None
gas_liberated_m3m3, gas_dissolved_m3m3 = [], []
gas_liberated_m3m3fs, gas_dissolved_m3m3fs = [], []
for p_bar in range(1, 120, 4):
    p_bar = p_bar + 0.1
    multiphase_flow.calc(p_bar, t_c)
    p_atm = uc.bar2atm(p_bar)
    
    rus_fluid_flow.calc(p_bar, t_c)
    rus_flow_model_db.get_data(rus_fluid_flow, object_name='rus_flow_model')
        
    python_flow_model_db.get_data(multiphase_flow, object_name='python_flow_model')
    vba_result = uniflocvba.MF_all_mf(p_atma = p_atm, **keywords_vba)
    
    this_gas_liberated_m3m3, this_gas_dissolved_m3m3 = calc_gas(multiphase_flow.fl.t_k,
                                                               multiphase_flow.fl.rho_oil_stkgm3,
                                                               multiphase_flow.fl.gamma_gas,
                                                               multiphase_flow.fl.p_mpa, 
                                                               multiphase_flow.fl.pb_bar / 10, 
                                                               multiphase_flow.fl.rsb_m3m3,
                                                               True)
    
    this_gas_liberated_m3m3fs, this_gas_dissolved_m3m3fs = calc_gas_with_fsolve(multiphase_flow.fl.t_k,
                                                               multiphase_flow.fl.rho_oil_stkgm3,
                                                               multiphase_flow.fl.gamma_gas,
                                                               multiphase_flow.fl.p_mpa, 
                                                               multiphase_flow.fl.pb_bar / 10, 
                                                               multiphase_flow.fl.rsb_m3m3,
                                                               True)
    gas_liberated_m3m3.append(this_gas_liberated_m3m3)
    gas_dissolved_m3m3.append(this_gas_dissolved_m3m3)
    gas_liberated_m3m3fs.append(this_gas_liberated_m3m3fs)
    gas_dissolved_m3m3fs.append(this_gas_dissolved_m3m3fs)
    
    this_vba_result_df = create_result_df_from_vba_output(vba_result, p_bar, t_c)
    try:
        vba_result_df = vba_result_df.append(this_vba_result_df)
    except:
        vba_result_df = this_vba_result_df.copy()

Объединение результатов

In [ ]:
result_unifloc_python = python_flow_model_db.get_data_as_df()
result_unifloc_python.index = result_unifloc_python['python_flow_model.p_bar']
result_unifloc_python.index.name = 'p_bar'


result_ru = rus_flow_model_db.get_data_as_df()
result_ru.index = result_ru['rus_flow_model.p_bar']
result_ru.index.name = 'p_bar'
result_ru = result_ru.add_prefix('ru_')

vba_result_df.index = result_unifloc_python.index
vba_result_df.index.name = 'p_bar'
vba_result_df['mn_kgsec'] = vba_result_df['mw_kgsec'] + vba_result_df['mg_kgsec'] + vba_result_df['mo_kgsec']
vba_result_df['liquid_content_d'] = 1 - vba_result_df['gas_fraction_d']
vba_result_df = vba_result_df.add_prefix('vba_flow_model.')
all_result = result_unifloc_python.join(vba_result_df)
df = pd.DataFrame({'gas_liberated_m3m3m':gas_liberated_m3m3,
                   'gas_dissolved_m3m3m':gas_dissolved_m3m3,
                  'gas_liberated_m3m3mfs':gas_liberated_m3m3fs,
                   'gas_dissolved_m3m3mfs':gas_dissolved_m3m3fs},
                   all_result.index)
all_result = all_result.join(df)
all_result = all_result.join(result_ru)

Построение графиков

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['dissolved_gas_sc_m3m3'],
                                              ['liberated_gas_sc_m3m3'],
                                              ['m3m3m'],
                                               ['gas_liberated_m3m3'],
                                              ['ru', 'rs_m3m3']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                 plot_name='Объем растворенного и выделившегося газа', 
                                x_name = 'Давление, бар', 
                                y_name = 'Объем газа, м3/м3',
                                 traces_names = ['Объем растворенного газа по Стендингу, м3/м3', 
                                                'Объем выделившегося газа по Стендингу, м3/м3', 
                                                'Объем выделившегося газа по Мищенко, м3/м3',
                                                'Объем растворенного газа по Мищенко, м3/м3']
                                 )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['gas_fraction'], ['liquid_content'],
                                              ], ['vba','fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  
                                plot_name='Объемная доля жидкости и газа', 
                                x_name = 'Давление, бар', 
                                y_name = 'д.ед.',
                                 traces_names = ['Объемная доля газа в потоке ГЖС, д.ед.', 
                                                'Объемная доля жидкости в потоке ГЖС, д.ед.' 
                                                ])

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['pb_bar'], ['cor_number'], swap_xy=False, reversed_y=False,  
                                 plot_name='Давление насыщения', 
                                x_name = 'Давление, бар',
                                y_name = 'Давление насыщения, бар',
                                traces_names = ['Давление насыщения по Стендингу, бар', 
                                                'Давление насыщения по исходным данным, бар'])

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mu_gas_cp']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,
                                 plot_name='Вязкость газа', 
                                 x_name = 'Давление, бар',
                                y_name = 'Вязкость газа, сПуаз',
                                #traces_names = ['Давление насыщения по Стендингу, бар', 
                                #                'Вязкость нефти, сПуаз']
                        )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['z']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,
                                 plot_name='Коэффициент сверхсжимаемости', 
                                 x_name = 'Давление, бар',
                                y_name = 'Коэффициент сверхсжимаемости',
                                traces_names = ['Давление насыщения по Стендингу, бар', 
                                                'Вязкость нефти, сПуаз']
                        )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['b_oil']], ['cor_number', 'cal', 'stkgm', 'b_oil_b'], swap_xy=False, reversed_y=False,
                                 plot_name='Объемный коэффициент нефти', 
                                 x_name = 'Давление, бар',
                                y_name = 'Объемный коэффициент нефти, м3/м3',
                                #traces_names = ['Давление насыщения по Стендингу, бар', 
                                #                'Вязкость нефти, сПуаз']
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['flow_model.mu'],['fl.mu']], ['dead','vba','oil_b','cor_number'], swap_xy=False, reversed_y=False, 
                                plot_name='Вязкость ГЖС и составляющих', 
                                x_name = 'Давление, бар', 
                                y_name = 'Вязкость, сПуаз',
                                 traces_names = ['Вязкость жидкости, сПуаз', 
                                                'Объемная вязкость ГЖС, сПуаз', 
                                                'Вязкость нефти, сПуаз',
                                                'Вязкость газа, сПуаз',
                                                'Вязкость воды, сПуаз']
                                 )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['flow_model.rho'],['fl.rho']], ['dead','vba','oil_b','cor_number', 'stkgm'], swap_xy=False, reversed_y=False, 
                                plot_name='Плотность ГЖС и составляющих', 
                                x_name = 'Давление, бар', 
                                y_name = 'Плотность, кг/м3',
                                traces_names = ['Плотность жидкости, кг/м3', 
                                               'Объемная плотность ГЖС, кг/м3', 
                                               'Плотность нефти, кг/м3',
                                               'Плотность газа, кг/м3',
                                               'Плотность воды, кг/м3']
                                
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['rho'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["heatcap"], ["cliq_JkgC"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["sigma"]], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["kgsec"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

# Суммарный массовый расход смеси должен быть постоянным

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'oil']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'wat']], ['fl.','cor_number', 'mass_fraction', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'gas']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rs'], ['liberated_gas'], ['dissolved_gas_sc_m3m3']], ['cor_number', 'cal', 'rsw'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['liq', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mix', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  ['fw'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['liquid_content']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rs']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rho_oil']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['b_oil']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mu_oil_cp']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')